In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Classical shadows
=================

::: {.meta}
:property=\"og:description\": Learn how to construct classical shadows
and use them to estimate observables. :property=\"og:image\":
<https://pennylane.ai/qml/_images/atom_shadow.png>
:::

::: {.related}
tutorial\_measurement\_optimize Measurement optimization quantum\_volume
Quantum volume tutorial\_quantum\_metrology Variationally optimizing
measurement protocols
:::

*Authors: Roeland Wiersema and Brian Doolittle (Xanadu Residents) ---
Posted: 14 June 2021. Last updated: 14 June 2021.*

![](../demonstrations/classical_shadows/atom_shadow.png){.align-center
width="75.0%"}

Estimating properties of unknown quantum states is a key objective of
quantum information science and technology. For example, one might want
to check whether an apparatus prepares a particular target state, or
verify that an unknown system is entangled. In principle, any unknown
quantum state can be fully characterized by [quantum state
tomography](https://arxiv.org/pdf/quant-ph/0302028.pdf). However, this
procedure requires accurate expectation values for a set of observables
whose size grows exponentially with the number of qubits. A potential
workaround for these scaling concerns is provided by the classical
shadow approximation introduced in a recent paper by Huang et al..

The approximation is an efficient protocol for constructing a *classical
shadow* representation of an unknown quantum state. The classical shadow
can be used to estimate properties such as quantum state fidelity,
expectation values of Hamiltonians, entanglement witnesses, and
two-point correlators.

![(Image from Huang et
al..)](../demonstrations/classical_shadows/classical_shadow_overview.png){.align-center
width="90.0%"}

In this demo, we use PennyLane to obtain classical shadows of a quantum
state prepared by a quantum circuit, and use them to reconstruct the
state and estimate expectation values of observables.


Constructing a Classical Shadow
===============================

Classical shadow estimation relies on the fact that for a particular
choice of measurement, we can efficiently store snapshots of the state
that contain enough information to accurately predict linear functions
of observables. Depending on what type of measurements we choose, we
have an information-theoretic bound that allows us to control the
precision of our estimator.

Let us consider an $n$-qubit quantum state $\rho$ (prepared by a
circuit) and apply a random unitary $U$ to the state:

$$\rho \to U \rho U^\dagger.$$

Next, we measure in the computational basis and obtain a bit string of
outcomes $|b\rangle = |0011\ldots10\rangle$. If the unitaries $U$ are
chosen at random from a particular ensemble, then we can store the
reverse operation $U^\dagger |b\rangle\langle b| U$ efficiently in
classical memory. We call this a *snapshot* of the state. Moreover, we
can view the average over these snapshots as a measurement channel:

$$\mathbb{E}\left[U^\dagger |b\rangle\langle b| U\right] = \mathcal{M}(\rho).$$

If the ensemble of unitaries defines a tomographically complete set of
measurements, we can invert the channel and reconstruct the state:

$$\rho = \mathbb{E}\left[\mathcal{M}^{-1}\left(U^\dagger |b\rangle\langle b| U \right)\right].$$

If we apply the procedure outlined above $N$ times, then the collection
of inverted snapshots is what we call the *classical shadow*

$$S(\rho,N) = \left\{\hat{\rho}_1= \mathcal{M}^{-1}\left(U_1^\dagger |b_1\rangle\langle b_1| U_1 \right)
,\ldots, \hat{\rho}_N= \mathcal{M}^{-1}\left(U_N^\dagger |b_N\rangle\langle b_N| U_N \right)
\right\}.$$

The inverted channel is not physical, i.e., it is not completely postive
and trace preserving (CPTP). However, this is of no concern to us, since
all we care about is efficiently applying this inverse channel to the
observed snapshots as a post-processing step.

Since the shadow approximates $\rho$, we can now estimate **any**
observable with the empirical mean:

$$\langle O \rangle = \frac{1}{N}\sum_i \text{Tr}{\hat{\rho}_i O}.$$

Note that the classical shadow is independent of the observables we want
to estimate, as $S(\rho,N)$ contains only information about the state!

Furthermore, the authors of prove that with a shadow of size $N$, we can
predict $M$ arbitary linear functions
$\text{Tr}{O_1\rho},\ldots,\text{Tr}{O_M \rho}$ up to an additive error
$\epsilon$ if
$N\geq \mathcal{O}\left(\log{M} \max_i ||O_i||^2_{\text{shadow}}/\epsilon^2\right)$.
The shadow norm $||O_i||^2_{\text{shadow}}$ depends on the unitary
ensemble that is chosen.

Two different ensembles can be considered for selecting the random
unitaries $U$:

1.  Random $n$-qubit Clifford circuits.
2.  Tensor products of random single-qubit Clifford circuits.

Although ensemble 1 leads to the most powerful estimators, it comes with
serious practical limitations since $n^2 / \log(n)$ entangling gates are
required to sample the Clifford circuit. The snapshots of both ensembles
can be stored efficiently using the [stabilizer
formalism](https://arxiv.org/abs/quant-ph/9705052). Single-qubit
Clifford circuits rotate the measurement basis to one of the Pauli
eigenbases, so ensemble 2 is equivalent to measuring single shots of
single-qubit Pauli observables on all qubits. For the purposes of this
demo we focus on ensemble 2, which is a more NISQ-friendly approach.

This ensemble comes with a significant drawback: the shadow norm
$||O_i||^2_{\text{shadow}}$ becomes dependent on the locality $k$ of the
observables that we want to estimate:

$$||O_i||^2_{\text{shadow}} \leq 4^k ||O_i||_\infty^2.$$

Say that we want to estimate the single expectation value of a Pauli
observable $\langle X_1 \otimes X_2 \otimes \ldots \otimes X_n \rangle$.
Estimating this from repeated measurements would require $1/\epsilon^2$
samples, whereas we would need an exponentially large shadow due to the
$4^n$ appearing in the bound. Therefore, classical shadows based on
Pauli measurements only offer an advantage when we have to measure a
large number of observables with modest locality.

We will now demonstrate how to obtain classical shadows using PennyLane.


A classical shadow is a collection of $N$ individual snapshots
$\hat{\rho}_i$. Each snapshot is obtained with the following procedure:

1.  The quantum state $\rho$ is prepared with a circuit.
2.  A unitary $U$ is randomly selected from the ensemble and applied to
    $\rho$.
3.  A computational basis measurement is performed.
4.  The snapshot is recorded as the observed eigenvalue $1,-1$ for
    $|0\rangle,|1\rangle$, respectively, and the index of the randomly
    selected unitary $U$.

To obtain a classical shadow using PennyLane, we design the
`calculate_classical_shadow` function below. This function obtains a
classical shadow for the state prepared by an input `circuit_template`.


As an example, we demonstrate how to use `calculate_classical_shadow`
and check its performance as the number of snapshots increases. First,
we will create a two-qubit device and a circuit that applies an `RY`
rotation to each qubit.


In [ ]:
# set up a two-qubit device with shots = 1 to ensure that we only get a single measurement



# simple circuit to prepare rho









# arrays in which to collect data



# collecting shadows and elapsed times









# printing out the smallest shadow as an example

Observe that the shadow simply consists of two matrices. Each qubit
corresponds to a different column. The first matrix describes outcome of
the measurement while the second matrix indexes the measurement applied
to each qubit. We now plot the computation times taken to acquire the
shadows.


As one might expect, the computation time increases linearly with the
number of snapshots. This linear scaling is useful for predicting the
length of time required to obtain a sufficient number of snapshots for
observable estimation.


State Reconstruction from a Classical Shadow
============================================

To verify that the classical shadow approximates the exact state that we
want to estimate, we tomographically reconstruct the original quantum
state $\rho$ from a classical shadow. Remember that we can approximate
$\rho$ by averaging over the snapshots and applying the inverse
measurement channel,

$$\rho = \mathbb{E}\left[\mathcal{M}^{-1}(U^{\dagger}|\hat{b}\rangle\langle\hat{b}|U)\right].$$

The expectation $\mathbb{E}[\cdot]$ describes the average over the
measurement outcomes $|b\rangle$ and the sampled unitaries. Inverting
the measurement channel may seem formidable at first, however, Huang et
al. show that for Pauli measurements we end up with a rather convenient
expression,

$$\rho=\mathbb{E}[\hat{\rho}], \quad \text{where} \quad
\hat{\rho} = \bigotimes_{j=1}^n(3U^{\dagger}_j|\hat{b}_j\rangle\langle\hat{b}_j|U_j-\mathbb{I}).$$

Here $\hat{\rho}$ is a snapshot state reconstructed from a single sample
in the classical shadow, and $\rho$ is the average over all snapshot
states $\hat{\rho}$ in the shadow.

To implement the state reconstruction of $\rho$ in PennyLane, we develop
the `shadow_state_reconstruction` function.


Example: Reconstructing a Bell State
====================================

First, we construct a single-shot, `'default.qubit'` device and define
the `bell_state_circuit` QNode to construct and measure a Bell state.


In [ ]:
# circuit to create a Bell state and measure it in
# the bases specified by the 'observable' keyword argument.

Then, construct a classical shadow consisting of 1000 snapshots.


To reconstruct the Bell state we use `shadow_state_reconstruction`.


Note the resemblance to the exact Bell state density matrix.


To measure the closeness we can use the operator norm.


In [ ]:
# Calculating the distance between ideal and shadow states.

Finally, we see how the approximation improves as we increase the number
of snapshots. We run the estimator 10 times for each $N$.


In [ ]:
# run the estimation multiple times so that we can include error bars

As expected, when the number of snapshots increases, the state
reconstruction becomes closer to the ideal state.


Estimating Pauli Observables with Classical Shadows
===================================================

We have confirmed that classical shadows can be used to reconstruct the
state. However, the goal of classical shadows is not to perform full
tomography, which takes an exponential amount of resources. Instead, we
want to use the shadows to efficiently calculate linear functions of a
quantum state. To do this, we write a function
`estimate_shadow_observable` that takes in the previously constructed
shadow $S(\rho, N)=[\hat{\rho}_1,\hat{\rho}_2,\ldots,\hat{\rho}_N]$, and
estimates any observable via a median of means estimation. This makes
the estimator more robust to outliers and is required to formally prove
the aforementioned theoretical bound. The procedure is simple: split up
the shadow into $K$ equally sized chunks and estimate the mean for each
of these chunks,

$$\langle O_{(k)}\rangle = \text{Tr}\{O \hat{\rho}_{(k)}\} \quad
\text{and} \quad \hat{\rho}_{(k)} = \frac{1}{ \lfloor N/K \rfloor }
\sum_{i=(k-1)\lfloor N/K \rfloor + 1}^{k \lfloor N/K \rfloor } \hat{\rho}_i.$$

The median of means estimator is then simply the median of this set

$$\langle O\rangle = \text{median}\{\langle O_{(1)} \rangle,\ldots, \langle O_{(K)} \rangle \}.$$

Note that the shadow bound has a failure probability $\delta$. By
choosing the number of splits $K$ to be suitably large, we can
exponentially suppress this failure probability. Assume now that
$O=\bigotimes_j^n P_j$, where $P_j \in \{I, X, Y, Z\}$. To efficiently
calculate the estimator for $O$, we look at a single snapshot outcome
and plug in the inverse measurement channel:

$$\begin{aligned}
\text{Tr}\{O\hat{\rho}_i\} &= \text{Tr}\{\bigotimes_{j=1}^n P_j (3U^{\dagger}_j|\hat{b}_j\rangle\langle\hat{b}_j|U_j-\mathbb{I})\}\\
 &= \prod_j^n \text{Tr}\{ 3 P_j U^{\dagger}_j|\hat{b}_j\rangle\langle\hat{b}_j|U_j\}.
\end{aligned}$$

Due to the orthogonality of the Pauli operators, this evaluates to
$\pm 3$ if $P_j$ is the corresponding measurement basis $U_j$ and 0
otherwise. Hence if a single $U_j$ in the snapshot does not match the
one in $O$, the whole product evaluates to zero. As a result,
calculating the mean estimator can be reduced to counting the number of
exact matches in the shadow with the observable, and multiplying with
the appropriate sign. Below, we develop the function
`estimate_shadow_obervable` to estimate any observable given a classical
shadow.


Next, we can define a function that calculates the number of samples
required to get an error $\epsilon$ on our estimator for a given set of
observables.


Example: Estimating a simple set of observables
===============================================

Here, we give an example for estimating multiple observables on a
10-qubit circuit. We first create a simple circuit


Next, we define our set of observables

$$O = \sum_{i=0}^{n-1} X_i X_{i+1} + Y_i Y_{i+1} + Z_i Z_{i+1}.$$


With the `shadow_bound` function, we calculate how many shadows we need
to ensure that the absolute error of all individual terms in $O$
satisfies

$$|\langle{O_i}\rangle_{shadow} - \langle{O_i}\rangle_{exact}| \leq \epsilon$$

for all $1\leq i \leq M$.


We verify the bound by considering a grid of errors $\epsilon_i$ and
checking that
$|\langle{O_i}\rangle_{shadow} - \langle{O_i}\rangle_{exact}|$ stays
below this value for the shadow size calculated in `shadow_bound`.
First, we get the classical shadow estimate.


In [ ]:
# create a grid of errors

Then, we calculate the ground truth by changing the device backend.


In [ ]:
# change the simulator to be the exact one.

Finally, we plot the errors
$|\langle{O_i}\rangle_{shadow} - \langle{O_i}\rangle_{exact}|$ for all
individual terms in $O$. We expect that these errors are always smaller
than $\epsilon$.


The points in the plot indicate the individual errors for all $O_i$ at a
given shadow size. The dashed line represents the error threshold that
these points must stay under to satisfy the bound. As expected, the
bound is satisfied for all $O_i$ and the errors decrease with the size
of the shadow.

To conclude, we have shown that classical shadows can be used to
reconstruct quantum states and estimate expectation values of
observables. This is but the tip of the iceberg of what is possible with
this technique. In the original work, the authors estimate fidelities,
calculate entanglement witnesses, and even find a way to approximate the
von Neumann entropy. These applications illustrate the potential power
of classical shadows for the characterization of quantum systems.


References
==========

About the authors
=================
